In [140]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [539]:
##LOAD UP THE DATA
## 0. check the curernt path, configured to run from /python_source
print(os.getcwd())

## 1. define paths 
main_path = "../"
plot_path = main_path + '/plots/'
data_path = main_path + '/data/raw/core/'

##the specific subfolders check the ABCD data dictionary

mental_healht = data_path + '/abcd-general/'
gender_sexual_path = data_path + '/gender-identity-sexual-health/'
mental_health_path = data_path + '/mental-health/'



##2. check if the path exists
isExist = os.path.exists(plot_path)
isExist2 = os.path.exists(data_path)
print(f'plot paths correctly set: {isExist}, data path: {isExist2}')

##3. read up the csvs
gender_csvs = {}
general_csvs = {}
mental_health_csvs = {}
for csv in os.listdir(gender_sexual_path):
    gender_csvs[csv.split('.')[0]] = pd.read_csv(gender_sexual_path + csv)

for csv in os.listdir(general_path):
    general_csvs[csv.split('.')[0]] = pd.read_csv(general_path + csv)


for csv in os.listdir(mental_health_path):
    mental_health_csvs[csv.split('.')[0]] = pd.read_csv(mental_health_path + csv,dtype='unicode')


##3. look at the keys from the filenames
print(f'Keys in dictionary gender/sexual identity {gender_csvs.keys()}')
print(f'Keys in dictionary general {general_csvs.keys()}')
print(f'Keys in dictionary mental health {mental_health_csvs.keys()}')


#now we have the dictionary for all the csvs in abcd general !



/Users/samuelheczko/Documents/School/phd/mini-projects/Kelleher_block_2/code/python_source
plot paths correctly set: True, data path: True


/var/folders/5x/y5wv_9xn73sgkmjg4x4c3r3h0000gn/T/ipykernel_68280/3468429232.py:31: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  general_csvs[csv.split('.')[0]] = pd.read_csv(general_path + csv)


Keys in dictionary gender/sexual identity dict_keys(['gish_p_gi', 'gish_y_sex', 'gish_p_sex', 'gish_y_gi'])
Keys in dictionary general dict_keys(['abcd_p_demo', 'abcd_y_lt', 'abcd_y_lf', 'abcd_p_ocp', 'abcd_p_screen'])
Keys in dictionary mental health dict_keys(['mh_y_cbb', 'mh_y_le', 'mh_p_ksads_ss', 'mh_t_bpm', 'mh_p_ksads_sep', 'mh_y_ksads_cd', 'mh_p_ksads_gad', 'mh_p_ksads_ocd', 'mh_p_ksads_hi', 'mh_y_bisbas', 'mh_y_ksads_bg', 'mh_p_ksads_odd', 'mh_p_ksads_ptsd', 'mh_p_cbcl', 'mh_p_eatq', 'mh_p_ksads_phb', 'mh_y_ksads_ed', 'mh_y_ksads_si', 'mh_p_pss', 'mh_p_ksads_asd', 'mh_y_ksads_gad', 'mh_p_ksads_psy', 'mh_y_7up', 'mh_p_abcl', 'mh_p_ksads_bg', 'mh_p_ksads_bp', 'mh_y_pps', 'mh_p_ksads_ago', 'mh_p_ksads_dmdd', 'mh_y_peq', 'mh_p_ksads_sad', 'mh_y_ksads_bip', 'mh_y_ksads_dmdd', 'mh_p_ders', 'mh_p_ksads_slp', 'mh_p_ksads_cd', 'mh_p_ssrs', 'mh_y_ksads_dep', 'mh_y_ksads_ss', 'mh_y_erq', 'mh_p_asr', 'mh_p_fhx', 'mh_y_upps', 'mh_y_bpm', 'mh_y_ksads_sad', 'mh_y_ksads_slp', 'mh_p_ksads_dep'

In [540]:
##ADD SEX AT BIRTH + GENDER CONFORMITY TO THE DATAFRAME

dataframe_for_regression = pd.DataFrame() ##create empty dataframe to save the sex at birth and the gender conformity


subjects_gender_i = gender_csvs['gish_y_gi']['src_subject_id'].unique() ##extract subject codes in gender/sexuality dataset
subjects_demo = general_csvs['abcd_p_demo']['src_subject_id'].unique() ##extract subject codes in general geo dataset
subjects = np.array(list(set(subjects_demo) & set(subjects_gender_i))) ##define subjects that are present in both

gender_list = [] ##initieate arrays - for strings we need lists rather than np arrays
gender_i_list = np.zeros(len(subjects))
gender_number = np.zeros(len(subjects))

for subject_i, subject in enumerate(subjects):
    #print(subject)
    boolean_subject_gi = np.isin(gender_csvs['gish_y_gi']['src_subject_id'],[subject])
    boolean_subject_demo = np.isin(general_csvs['abcd_p_demo']['src_subject_id'],[subject])
    
    if general_csvs['abcd_p_demo'].iloc[boolean_subject_demo,:]['demo_sex_v2'].dropna().values == 1.0:
        gender_list.append('M') ##save the gender
        gender_number[subject_i] = 0 ##0.0 for males
        gender_i_list[subject_i] = gender_csvs['gish_y_gi'].iloc[boolean_subject_gi,:]['gish_m1_y'].dropna().mean() ##save the average feeling of belonign to male gender (higher - more conforming)
    else:
        gender_list.append('F')
        gender_number[subject_i] = 1 ##1.1 for females
        gender_i_list[subject_i] = gender_csvs['gish_y_gi'].iloc[boolean_subject_gi,:]['gish_f1_y'].dropna().mean()  ##save the average feeling of belonign to female gender (higher - more conforming)

dataframe_for_regression['Subject'] = subjects
dataframe_for_regression['Sex_at_birth'] = gender_list
dataframe_for_regression['Sex_numerical'] = gender_number
dataframe_for_regression['Gender_Conformity'] = gender_i_list


    

In [545]:
##ADD ADHD SYMPTOMS

##Initiate the arrays we need - in addition to what we had first we need to add a structure to account for all the events. And in additon we need to account that some time points are missing!

events = mental_health_csvs['mh_p_cbcl']['eventname'].unique() ##find all the events that are in the data
event_dict = {} ##intiate the dictionary
for possible_event in events:
    event_dict[possible_event] = [] ##all entries in the dictionary are lists - we will add elements later

#event_dict['Subject1'] = [] code for testing that we actually match the subject 


for subject_i, subject in enumerate(subjects): #for all subjects as found earlier

    score = mental_health_csvs['mh_p_cbcl'][mental_health_csvs['mh_p_cbcl']["src_subject_id"].isin([subject])][['eventname','cbcl_scr_dsm5_adhd_t']] ##find the subjects in the mental health dataframe (all events)
    #event_dict['Subject1'].append(subject) code for testing that we actually match the subject 

    for t in events: ##for all events
        event_subject_score = score[score['eventname'] == t]['cbcl_scr_dsm5_adhd_t'].values ##find the score of event and subject combo
        
        if len(event_subject_score) == 0: ##account for the fact that some events are missing
            event_subject_score = 0 ##if missing set score to 0
        else: 
            event_subject_score = event_subject_score[0]  ##if not missing use the score
        event_dict[t].append(event_subject_score)





## add the dictionary we created to the dataframe we made earlier (make a copy thats easier to change the code leading to it) add the ADHD perfix to keys we have
dataframe_for_regression_2 = pd.concat([dataframe_for_regression,pd.DataFrame(event_dict).add_prefix('ADHD_')],axis = 1)


In [547]:
##check our dataframe
dataframe_for_regression_2.head()

,Subject,Sex_at_birth,Sex_numerical,Gender_Conformity,ADHD_baseline_year_1_arm_1,ADHD_1_year_follow_up_y_arm_1,ADHD_2_year_follow_up_y_arm_1,ADHD_3_year_follow_up_y_arm_1,ADHD_4_year_follow_up_y_arm_1
0,NDAR_INV4LCR7BFU,M,0.0,4.0,53,50,50,51,0
1,NDAR_INVE367LYLC,M,0.0,5.0,50,53,50,50,50
2,NDAR_INVTELF8WZW,M,0.0,5.0,50,50,50,50,50
3,NDAR_INV2E00U6NB,F,1.0,5.0,50,50,51,51,0
4,NDAR_INVL3C0W7BT,F,1.0,5.0,50,50,50,50,0


In [491]:
event_dict

{'baseline_year_1_arm_1': array(['51'], dtype=object),
 '1_year_follow_up_y_arm_1': array([], dtype=object),
 '2_year_follow_up_y_arm_1': array(['58'], dtype=object),
 '3_year_follow_up_y_arm_1': array(['56'], dtype=object),
 '4_year_follow_up_y_arm_1': array([], dtype=object)}

In [428]:
for t in score['eventname']:
    print('ADHD_' + t)

ADHD_baseline_year_1_arm_1
ADHD_2_year_follow_up_y_arm_1
ADHD_3_year_follow_up_y_arm_1


In [209]:
print(gender_csvs['gish_y_gi']['src_subject_id'].unique().shape)
c.shape

(11866,)


(11868,)

In [178]:
dataframe_for_regression.iloc[np.isnan(dataframe_for_regression.gender_i_strenght).values,:]



,subject,birth_gender,gender_i_strenght
115,NDAR_INV09AEBLZH,M,NaN
126,NDAR_INV0A86UD86,M,NaN
231,NDAR_INV0JCWR24P,M,NaN
239,NDAR_INV0JWEE23L,M,NaN
262,NDAR_INV0LN1KD13,M,NaN
...,...,...,...
11676,NDAR_INVZF3HPJFM,M,NaN
11705,NDAR_INVZJ7JH5RE,M,NaN
11726,NDAR_INVZKDXUC63,M,NaN
11763,NDAR_INVZNTJH2GT,M,NaN


In [157]:
dataframe_for_regression.gender_i_strenght.nsmallest(10)

308      0.00
1562     0.00
1917     0.00
3776     0.00
3837     0.00
5558     0.00
6654     0.00
6965     0.00
7288     0.00
11603    0.75
Name: gender_i_strenght, dtype: float64

In [86]:
##MAKE THE DATAFRAME TO USE FOR THE REGRESSION 
dataframe_for_regression = pd.DataFrame()
gish_y_gi['gender']

gender_csvs

KeyError: 'gender'